In [2]:
# normalized VGG feature data concatenated with audio data into simple LSTM


# First, import necessary libraries
import tensorflow as tf
import numpy as np
import keras.backend as K

/Users/chloeloughridge/anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
Using TensorFlow backend.


In [3]:
# setting up the keras stuff
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, BatchNormalization
from keras.layers import LSTM
# my custom data_utils file
from data_utils_local08 import *

In [4]:
# uploading the X values
#X_input = load_Xinput(get_fc6_directory(7)) #VGG16 data

X_input = np.load('../Desktop/ResNet50_feature_data.npy')
print(X_input.shape)

(7, 212, 4096)


In [4]:
# uploading the Y values
y_data_input = fear_oneHot(212, 'fear_annotations_part01/MEDIAEVAL18_7_Fear.txt')
print(y_data_input.shape)

(212,)


In [5]:
# Uploading part01 of the local fc6 data

timesteps = 212   # the number of seconds in movie 07 --> i will figure out how to programmatically get this value
data_dim = 4096    # the number of output values from VGG16 layer fc6 --> switch to programmatic later
num_movies = 4
batch_size = 7
num_epochs = 5
validation_num = 3

# set up the X_train_data master array
X_train_data = np.zeros([num_movies, timesteps, data_dim]) #oooooh this array will have to be as long as the longest
                                                            # movie and padded with zeros --> this won't cause problems
                                                            # right?
X_valid_data = np.zeros([validation_num, timesteps, data_dim])
        
# for each movie number between and including 7 and 13
for num in range(num_movies):
    # load the X_input data
    X_input = load_Xinput(get_fc6_directory(7 + num))
    # put this X_input data into the X_train_data array
    X_train_data[num, :, :] = X_input
print(X_train_data.shape)



(4, 212, 4096)


In [6]:
# loading X validation set
X_valid_data = np.zeros([validation_num, timesteps, data_dim])

for num in range(validation_num):
    # load the X_input data
    X_valid = load_Xinput(get_fc6_directory(7 + num_movies + num))
    # put this X_input data into the X_train_data array
    X_valid_data[num, :, :] = X_valid
print(X_valid_data.shape)


(3, 212, 4096)


In [7]:
!pwd

/Users/chloeloughridge/AIGrant_LSTMdev


In [8]:
# uploading the y data

# set up y_train_data master array
Y_train_data = np.zeros([num_movies, timesteps])

# for each movie number between and including 7 and 13
for num in range(num_movies):
    # create the appropriate path to the fear annotation data
    #print(num)
    path = os.path.join('fear_annotations_part01/MEDIAEVAL18_{}_Fear.txt'.format(7+num))
    # create a one-hot vector
    y_data = fear_oneHot(timesteps, path)
    # add this one-hot vector to y_train_data
    Y_train_data[num, :] = y_data
print(Y_train_data[0])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [9]:
# upload the Y validation set
Y_valid_data = np.zeros([validation_num, timesteps])

# for each movie number in validation set
for num in range(validation_num):
    # create the appropriate path to the fear annotation data
    #print(num)
    path = os.path.join('fear_annotations_part01/MEDIAEVAL18_{}_Fear.txt'.format(7+ num_movies + num))
    # create a one-hot vector
    y_valid = fear_oneHot(timesteps, path)
    # add this one-hot vector to y_train_data
    Y_valid_data[num, :] = y_valid
print(Y_valid_data[0])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [10]:
# constructing a many-to-one LSTM model in keras --> inspiration: https://stackoverflow.com/questions/43034960/many-to-one-and-many-to-many-lstm-examples-in-keras
# i will start by training a model on only the VGG16 fc6 layer output (that's just one feature)
# should I eventually abstract this LSTM model? Create its own object file?
model = Sequential()

# normalization
model.add(BatchNormalization(input_shape=(timesteps, data_dim)))

model.add(LSTM(212, return_sequences=True))
#input_shape=(timesteps, data_dim)

# going to try adding a flatten layer in here
model.add(Flatten()) # I got this from a github thing, but I still don't completely understand why it works
# add the final dense layer and then softmax
model.add(Dense(212, activation='sigmoid'))
# going to add a softmax activation to this
#model.add(Activation('softmax'))

In [11]:
# a new metric of evaluation! the F-score!
def FScore2(y_true, y_pred):
    '''
    The F score, beta=2
    '''
    B2 = K.variable(4)
    OnePlusB2 = K.variable(5)
    pred = K.round(y_pred)
    tp = K.sum(K.cast(K.less(K.abs(pred - K.clip(y_true, .5, 1.)), 0.01), 'float32'), -1)
    fp = K.sum(K.cast(K.greater(pred - y_true, 0.1), 'float32'), -1)
    fn = K.sum(K.cast(K.less(pred - y_true, -0.1), 'float32'), -1)

    f2 = OnePlusB2 * tp / (OnePlusB2 * tp + B2 * fn + fp)

    return K.mean(f2)

In [12]:
# compiling LSTM model
# note that Ng used an Adam optimizer and categorical cross-entropy loss
# but this is a binary classification problem so I think the parameters below should suffice
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['binary_accuracy', FScore2])

In [13]:
# running the LSTM model
model.fit(X_train_data, Y_train_data, epochs = 20, validation_data=(X_valid_data, Y_valid_data))
print("finished training!")

Train on 4 samples, validate on 3 samples
Epoch 1/20
4/4 [==============================] - 6s 2s/step - loss: 0.7303 - binary_accuracy: 0.4847 - FScore2: 0.2761 - val_loss: 0.8384 - val_binary_accuracy: 0.6525 - val_FScore2: 0.1889
Epoch 2/20
4/4 [==============================] - 3s 846ms/step - loss: 0.0043 - binary_accuracy: 1.0000 - FScore2: 1.0000 - val_loss: 0.9081 - val_binary_accuracy: 0.6792 - val_FScore2: 0.1555
Epoch 3/20
4/4 [==============================] - 3s 836ms/step - loss: 7.4158e-04 - binary_accuracy: 1.0000 - FScore2: 1.0000 - val_loss: 1.1098 - val_binary_accuracy: 0.6494 - val_FScore2: 0.2210
Epoch 4/20
4/4 [==============================] - 3s 840ms/step - loss: 2.5083e-05 - binary_accuracy: 1.0000 - FScore2: 1.0000 - val_loss: 1.1106 - val_binary_accuracy: 0.6494 - val_FScore2: 0.2210
Epoch 5/20
4/4 [==============================] - 3s 835ms/step - loss: 1.6161e-05 - binary_accuracy: 1.0000 - FScore2: 1.0000 - val_loss: 1.1124 - val_binary_accuracy: 0.6557 -

In [14]:
# trying to view the model output
out = model.predict(X_train_data)
print("model prediction:")
print(out[0])
print("target:")
print(Y_train_data[0])

print("before 64:")
print(out[0][63])
print("64:")
print(out[0][65])

print("rounded")
print(np.round(out)[0])

model prediction:
[1.54748250e-05 3.73860348e-06 6.22802236e-06 6.84977522e-06
 5.97175756e-07 6.51306891e-06 9.21828178e-06 4.08543929e-06
 3.54271606e-06 4.45281330e-06 1.05653760e-06 1.66419741e-05
 5.98137831e-06 7.53025142e-06 9.62440481e-06 2.18011792e-07
 6.78542847e-06 4.69344695e-06 1.29463770e-05 1.11682666e-05
 2.12881332e-05 1.12767366e-05 1.87479200e-05 3.29778686e-07
 1.18505579e-06 1.20513505e-06 2.08084548e-05 1.91864228e-06
 1.13845972e-05 3.08246649e-06 1.25656243e-06 8.62167599e-06
 6.54126438e-07 1.17486788e-05 1.34551519e-05 2.64658092e-06
 2.64688128e-06 8.37437995e-08 1.31720908e-05 1.16092569e-05
 5.34302490e-06 1.08353806e-05 5.57745625e-06 4.13202042e-06
 8.79331492e-06 4.44126954e-06 5.46432875e-06 5.91790376e-06
 1.39598424e-05 1.18559842e-06 6.46799833e-07 2.81252378e-05
 5.32632248e-06 1.97402642e-05 2.62943504e-05 7.93019990e-06
 1.48234894e-05 3.32322852e-05 6.08982918e-06 3.26837471e-05
 1.08267659e-05 6.69205247e-06 2.06846398e-05 2.52684003e-07
 9.999

In [15]:
#try visualizing this model at some point?